In [1]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import subprocess
export_folder = "/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing"
from scipy.stats import ttest_ind
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LinearRegression
from lifelines.statistics import logrank_test
from tqdm import tqdm
from sklearn.cluster import AgglomerativeClustering

In [25]:
pheno_cols = ["ADP_FIB","TRAP_FIB"]
EUR_path_interval = "%s/genetic_score/EUR_interval" % export_folder
EUR_path = "%s/genetic_score/EUR" % export_folder

# Load GWAS

In [4]:
tag = "5mars"

In [5]:
gws = []
#for c in tqdm(pheno_cols):
for c in tqdm(pheno_cols):
    gw = pd.read_csv("%s/%s/bolt_res_feb_%s" % (export_folder,tag,c),sep="\t")
    gw["pheno"] = c
    gw.P_BOLT_LMM = gw.P_BOLT_LMM.astype(float)
    gws.append(gw)
gw = pd.concat(gws,axis=0,copy=False)
gw.rename(columns={"P_BOLT_LMM":"P"},inplace=True)

100%|██████████| 2/2 [00:22<00:00, 11.12s/it]


In [6]:
gw_ = gw.loc[gw.P < 1e-3]
gw_["POS"] = gw_["BP"]
for ch in range(1,23):
    gw_.loc[gw_.CHR > ch,"POS"] += gw.loc[gw.CHR == ch,"BP"].max()

/home/hv270/platelets/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/hv270/.local/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/hv270/.local/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [7]:
UKBB_SNPS = []
for chrom in tqdm(range(1,23)):
    with open("%s/UKBB_SNPS_CHR%d.snplist" % (export_folder, chrom)) as f:
        UKBB_SNPS += [c[:-1] for c in f.readlines()]

print(len(UKBB_SNPS))
print(UKBB_SNPS[:10])

gw_["IN_UKBB"] = gw_.SNP.isin(UKBB_SNPS)
print(gw_["IN_UKBB"].mean())

100%|██████████| 22/22 [01:20<00:00,  3.64s/it]


93095623
['rs367896724', 'rs540431307', 'rs201106462', 'rs548419688', 'rs568405545', 'rs534229142', 'rs537182016', 'rs572818783', 'rs538322974', '1:10616_CCGCCGTTGCAAAGGCGCGCCG_C']
0.9280581305265407


/home/hv270/platelets/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [15]:
causal_snps = {}
for c in pheno_cols:
    causal_path = "%s/conditional_analysis/causal_SNPs_%s_%s.txt" % (export_folder,tag,c)
    causal_snps[c] = [c[:-1] for c in open(causal_path,"r").readlines()]

In [16]:
good = {}
for c in pheno_cols:
    good[c] = gw.loc[(gw.pheno == c) & (gw.SNP.isin(causal_snps[c]))]

In [19]:
betas = good["ADP_FIB"][["SNP","BETA"]].rename(columns={"BETA":"ADP_FIB"})
for c in pheno_cols[1:]:
    betas = pd.merge(betas,good[c][["SNP","BETA"]].rename(columns={"BETA":c}),
                     left_on="SNP",right_on="SNP",how="outer")
betas = pd.merge(betas,gw.loc[gw.pheno == "ADP_FIB",["SNP","ALLELE1"]],left_on="SNP",right_on="SNP",how="left")
betas = betas[["SNP","ALLELE1"] + pheno_cols]
betas.fillna(0,inplace=True)
assert betas.shape[0] == betas.SNP.nunique()

In [21]:
good_chroms = []
for c in good:
    good_chroms += good[c].CHR.unique().tolist()
good_chroms = list(set(good_chroms))
print(good_chroms)

[1, 3, 5, 7, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 20]


In [26]:
betas_path = "%s/Genetic_score_Sysmex_traits/betas.csv" % (export_folder)
betas.to_csv(betas_path,
            sep="\t",index=False)

n_splits = 16

all_causal_path = "%s/conditional_analysis/causal_SNPs_%s_all.txt" % (export_folder,tag)

for chrom in good_chroms:
    command = "/home/hv270/Plink2/plink2"
    command += " --bgen /rds/project/wja24/rds-wja24-uk-biobank-gen/bgen/ukb_imp_chr%s_v3.bgen" % chrom
    command += " --sample /rds/project/wja24/rds-wja24-uk-biobank-gen/13745_specific/sample/ukb13745_imp_chr%s_v3_s487378.sample" % chrom
    #command += " --extract %s" % causal_snps_path
    command += " --keep %s" % EUR_path
    command += " --extract %s" % all_causal_path
    command += " --score %s header-read" % betas_path
    command += " --score-col-nums 3-%d" % (2 + len(pheno_cols))
    command += " --out %s/Genetic_score_Sysmex_traits/%s/good_score_chr%s" % (export_folder,tag,chrom)
    command += " --threads %d" % n_splits

    f_name = "/home/hv270/interval_slurm_calls/call_gs_chr_%s_%s.sh" % (chrom,tag)
    f = open(f_name,"w+")
    f.write("#!/bin/sh\n")

    f.write("#SBATCH -J chr%s\n" % (chrom))
    f.write("#SBATCH -A ASTLE-SL2-CPU\n")
    f.write("#SBATCH -N 1\n")
    f.write("#SBATCH -n %d\n" % n_splits)
    f.write("#SBATCH --time=8:30:00\n")
    f.write("#SBATCH -p skylake-himem\n")
    f.write("#SBATCH --output=/home/hv270/slurm_log/%s_%s_%s.out\n" % ("GS",tag,chrom))
    f.write("export OMP_NUM_THREADS=%d\n" % n_splits)

    f.write(". /etc/profile.d/modules.sh\n")
    f.write("module purge\n")
    f.write("module load rhel7/default-peta4\n")
    f.write(command)
    f.close()

    print("Submitting %s" % (chrom))
    p = subprocess.Popen("sbatch %s" % f_name,shell=True)
    p.wait()

Submitting 1
Submitting 3
Submitting 5
Submitting 7
Submitting 8
Submitting 9
Submitting 10
Submitting 11
Submitting 12
Submitting 15
Submitting 16
Submitting 17
Submitting 18
Submitting 19
Submitting 20
